In [15]:
import requests 
from bs4 import BeautifulSoup
import lxml

In [16]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

In [17]:
url = "https://www.zillow.com/homes/for_rent/1-_beds/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3Anull%2C%22mapBounds%22%3A%7B%22west%22%3A-122.69219435644531%2C%22east%22%3A-122.17446364355469%2C%22south%22%3A37.703343724016136%2C%22north%22%3A37.847169233586946%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22max%22%3A3000%7D%2C%22price%22%3A%7B%22max%22%3A872627%7D%2C%22beds%22%3A%7B%22min%22%3A1%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D"
header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.26",
         "Accept-Language":"en-US,en;q=0.9"}
response = requests.get(url, headers = header).text

In [18]:
Soup = BeautifulSoup(response, 'html.parser')
Soup.title

<title>Rental Listings - 1498 Rentals | Zillow</title>

In [19]:
all_links = []
all_link_elements = Soup.select('#grid-search-results li div a' )
for link in all_link_elements:
    if 'http' not in link['href']:
        all_links.append(f"https://www.zillow.com{link['href']}")
    else:
        all_links.append(link['href'])   
all_links = list(set(all_links))

In [20]:
all_address = []
all_address_elements = Soup.select('#grid-search-results li a address')
for address in all_address_elements:
    if '|' in address.text:
        all_address.append(address.text.split('|')[-1])
    elif '#' in address.text:
        all_address.append(address.text.split('#')[-1])

In [26]:
all_prices = []
all_price_elements = Soup.select('#grid-search-results span')
for price in all_price_elements:
    if price.text:
        if '$' in price.text:
            if '+ 2' not in price.text:
                if '/' in price.text:
                    all_prices.append(price.text.split('/')[0])
                else:
                    all_prices.append(price.text.split('+')[0])

In [27]:
print(len(all_address), len(all_links), len(all_prices))

9 9 9


In [28]:
print(all_prices)

['$2,095', '$2,404', '$2,195', '$2,326', '$2,475', '$2,738', '$2,400', '$2,250', '$2,487']


### Creating spreadsheet using Google Forms

In [32]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for i in range(len(all_links)):

    driver.get("https://docs.google.com/forms/d/e/1FAIpQLSdTUFpcg0c1pC-J1fLV-itFoMsRWgIGykyXijXfOxyXsXCxXw/viewform?usp=sf_link")
    time.sleep(1)
    
    question_1 = driver.find_element(by='xpath', value='//*[@id="mG61Hd"]/div[2]/div/div[2]/div[1]/div/div/div[2]/div/div[1]/div/div[1]/input')
    question_1.send_keys(all_address[i])
    
    question_2 = driver.find_element(by='xpath', value='//*[@id="mG61Hd"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/input')
    question_2.send_keys(all_prices[i])
    
    question_3 = driver.find_element(by='xpath', value='//*[@id="mG61Hd"]/div[2]/div/div[2]/div[3]/div/div/div[2]/div/div[1]/div/div[1]/input')
    question_3.send_keys(all_links[i])
    
    driver.find_element(by='xpath', value = '//*[@id="mG61Hd"]/div[2]/div/div[3]/div[1]/div[1]/div/span/span').click()

driver.quit()